In [ ]:
pip install -r requirements.txt 

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#imports
from bs4 import BeautifulSoup
import requests
import pandas as pd
import urllib.parse
import numpy as np

In [ ]:
url = 'https://formds.com/filings/newest'
response = requests.get(url)

In [ ]:
response.status_code

200

In [ ]:
soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
#check results
soup

<!DOCTYPE html>

<html lang="en">
<head>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>FormDs.com - fund raising filing</title>
<meta content="Newest Form D filings. Most recent fund raising on May 6, 2022" name="description"/>
<script src="https://ajax.googleapis.com/ajax/libs/jquery/3.5.1/jquery.min.js"></script>
<link href="/stylesheets/bootstrap/bootstrap.css" media="screen" rel="stylesheet"/>
<script crossorigin="anonymous" integrity="sha384-Tc5IQib027qvyjSMfHjOMaLkfuWVxZxUPnCJA7l2mCWNIpG9mGCD8wGNIcPD7Txa" src="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/js/bootstrap.min.js"></script>
<!--[if lte IE 9]>
  <script type="text/javascript">
  window.location = '/static/not_supported.html';
  </script>
<![endif]-->
<script type="text/javascript">

	var _gaq = _gaq || [];
	_gaq.push(['_setAccount', 'UA-8796308-7']);
	_gaq.push(['_trackPageview']);

	(function() {
		var ga = document.createElement('script'); ga.type = 'text/javascript'; ga.async = true;

In [ ]:
#pull company names and store in a list, excluding words that are not needed
not_comps = [' FormDs.com ', 'Newest', 'Filter', 'Local', '← Previous', 'Next →']
companies = [company.get_text() for company in soup.find_all('a')]
orgs = [company for company in companies if company not in not_comps]

In [ ]:
#check results
orgs

['EQ Fund I, a series of Climate Capital Deals, LP',
 'ELIZABETH PARK EVENT DRIVEN FUND, L.P.',
 'ELIZABETH PARK CAPITAL II L.P.',
 'BWR Innovations LLC',
 'ELIZABETH PARK CAPITAL, L.P.',
 'B2Digital, Inc.',
 'Elizabeth Park Capital Offshore Fund, Ltd.',
 'PLEOPHARMA, INC.',
 'RET Fund I, a series of Climate Capital Deals, LP',
 'CAPW Gaingels Fund I, a series of Riverside Ventures, LP',
 'Paradise Square Broadway Limited Partnership',
 'iCapital Renaissance Strategies (RIEF), L.P.',
 'Wesana Health Holdings Inc',
 'Your Problem, My Solution, LLC',
 'TRANSFORMATIVEMED INC.',
 'TH Fund I, a series of Roosh Ventures, LP',
 'Fund I, a series of Aditya Nagarsheth Funds, LP',
 'BeYu Labs LLC',
 'Stone-Goff Partners IV FTE, LP',
 'LCM HEALTHCARE FUND II, LP',
 'Accern Corp',
 'Independence SS LLC',
 'Zora Labs, Inc.',
 'Tri Peaks Capital Partners, LP',
 'Stone-Goff Partners IV, LP',
 'Cybera Global, Inc.',
 'Nuo Therapeutics, Inc.',
 'AD Fund II, a series of Palumni VC, LP',
 'PA1 Fund I, a 

In [ ]:
#pulls our funding round amount and stores in list
funding = [amount.get_text() for amount in soup.find_all('td', {'class': 'hidden-xs'})[0::3]]

In [ ]:
#check results
funding

['$220,007',
 '$132,554,674',
 '$73,099,558',
 '$200,000',
 '$76,057,501',
 '$270,480',
 '$203,455,329',
 '$11,122,252',
 '$119,000',
 '$137,416',
 '$13,500,000',
 '$55,375,656',
 '$485,670',
 'Yet to Sell',
 '$1,200,000',
 '$1,162,500',
 '$1,838,000',
 '$45,000',
 'Yet to Sell',
 'Yet to Sell',
 '$18,875,748',
 '$500,000',
 '$41,755,736',
 'Yet to Sell',
 'Yet to Sell',
 '$4,399,913',
 '$3,550,000',
 '$179,594',
 '$125,000',
 '$1,697,212']

In [ ]:
#pulls out the incremental cash amount which is the new amount of cash raised and stores it in a list
new_cash_raised = [cash.get_text() for cash in soup.find_all('td')[3::6]]

In [ ]:
#check results
new_cash_raised

['$220,007',
 '$18,645,558',
 '$0',
 '$35,000',
 '$2,121,717',
 '$270,480',
 '$750,000',
 '$11,122,252',
 '$119,000',
 '$137,416',
 '$0',
 '$3,817,728',
 '$485,670',
 '$0',
 '$1,200,000',
 '$1,162,500',
 '$1,838,000',
 '$45,000',
 '$0',
 '$0',
 '$18,875,748',
 '$500,000',
 '$41,755,736',
 '$0',
 '$0',
 '$4,399,913',
 '$3,550,000',
 '$179,594',
 '$125,000',
 '$1,697,212']

In [ ]:
#pull dates and store in a list
dates = [date.get_text() for date in soup.find_all('td', {'class': 'hidden-xs'})[1::3]]

In [ ]:
#check results
dates

['2022-05-06',
 '2022-05-06',
 '2022-05-06',
 '2022-05-06',
 '2022-05-06',
 '2022-05-06',
 '2022-05-06',
 '2022-05-06',
 '2022-05-06',
 '2022-05-06',
 '2022-05-06',
 '2022-05-06',
 '2022-05-06',
 '2022-05-06',
 '2022-05-06',
 '2022-05-06',
 '2022-05-06',
 '2022-05-06',
 '2022-05-06',
 '2022-05-06',
 '2022-05-06',
 '2022-05-06',
 '2022-05-06',
 '2022-05-06',
 '2022-05-06',
 '2022-05-06',
 '2022-05-06',
 '2022-05-06',
 '2022-05-06',
 '2022-05-06']

In [ ]:
#pull status or whether this is a new/amended filing
new_amend = [nwstatus.get_text() for nwstatus in soup.find_all('td', {'class': 'hidden-xs'})[2::3]]

In [ ]:
#check results
new_amend

['New Filing',
 'Amended',
 'Amended',
 'Amended',
 'Amended',
 'New Filing',
 'Amended',
 'New Filing',
 'New Filing',
 'New Filing',
 'Amended',
 'Amended',
 'New Filing',
 'New Filing',
 'New Filing',
 'New Filing',
 'New Filing',
 'New Filing',
 'New Filing',
 'New Filing',
 'New Filing',
 'New Filing',
 'New Filing',
 'New Filing',
 'New Filing',
 'New Filing',
 'New Filing',
 'New Filing',
 'New Filing',
 'New Filing']

In [ ]:
#bc stored data variables are lists, we need to tranpose, create a dataframe, and the columns for a complete dataframe
#essentially, this code stores the data in a dataframe after transposing the list(i.e final_results) of lists(i.e. orgs, funding, new_cash_raised, dates, and new_amended) to columns
final_results = [orgs, funding, new_cash_raised, dates, new_amend]
df = pd.DataFrame(final_results).transpose()
df.columns = ['Company Name', 'Funding Amount', 'New Cash Raised', 'Date', 'New or Amended'] 

In [ ]:
#check dataframe
df

,Company Name,Funding Amount,New Cash Raised,Date,New or Amended
0,"EQ Fund I, a series of Climate Capital Deals, LP","$220,007","$220,007",2022-05-06,New Filing
1,"ELIZABETH PARK EVENT DRIVEN FUND, L.P.","$132,554,674","$18,645,558",2022-05-06,Amended
2,ELIZABETH PARK CAPITAL II L.P.,"$73,099,558",$0,2022-05-06,Amended
3,BWR Innovations LLC,"$200,000","$35,000",2022-05-06,Amended
4,"ELIZABETH PARK CAPITAL, L.P.","$76,057,501","$2,121,717",2022-05-06,Amended
5,"B2Digital, Inc.","$270,480","$270,480",2022-05-06,New Filing
6,"Elizabeth Park Capital Offshore Fund, Ltd.","$203,455,329","$750,000",2022-05-06,Amended
7,"PLEOPHARMA, INC.","$11,122,252","$11,122,252",2022-05-06,New Filing
8,"RET Fund I, a series of Climate Capital Deals, LP","$119,000","$119,000",2022-05-06,New Filing
9,"CAPW Gaingels Fund I, a series of Riverside Ve...","$137,416","$137,416",2022-05-06,New Filing


In [ ]:
#convert dataframe to a csv
df.to_csv('new_funding_rounds.csv')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=422ecbd0-6b22-4ded-8d11-ea8c5e309476' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>